In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from pathlib import Path

In [2]:
df = pd.read_csv("data/nypd/allegations_202007271729.csv")

disp_map = {
    "Unsubstantiated": "Unsubstantiated",
    "Exonerated": "Exonerated",
}
df = df[df["year_received"] < 2020]

df["board_disposition"] = df["board_disposition"].map(lambda x: disp_map.get(x, "Substantiated"))

In [3]:
dispos = (
    df
    .groupby(["year_received", 'board_disposition'])
    [["unique_mos_id"]]
    .count()
    .reset_index()
    .rename(columns = {"unique_mos_id":"count"})
)
dispos["total"] = dispos.groupby("year_received")['count'].transform("sum")
dispos["percent"] = dispos["count"] * 100 / dispos["total"]

In [60]:
df_sub = dispos[dispos["board_disposition"] == "Substantiated"]

df_unsub = (
    dispos
    [dispos["board_disposition"] != "Substantiated"]
    .groupby("year_received")
    [["count", "percent"]]
    .sum()
    .reset_index()
)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_sub["year_received"],
    y=df_sub["percent"],
    name="Misconduct Found %",
))

fig.add_trace(go.Bar(
    x=df_unsub["year_received"],
    y=df_unsub["count"],
    name="No Misconduct Found",
    hovertemplate="In %{x}, there were %{y} unsubstantiated complaints.<extra></extra>"
))

fig.update_layout(
    xaxis=dict(title="Year",),
    #yaxis=dict(tickvals=list(range(0,1800,300))),
    legend=dict(title="Complaints:",
                x=0.02, xanchor='left', xref='paper',
                y=0.7, yanchor='bottom', yref='paper',
                bgcolor="rgba(255,255,255,.7)",),
    font=dict(size=18, family="Arial, sans-serif"),
    barmode="stack",
    autosize=True,
    height = 670,
    width = 1200,
    colorway = pio.templates["plotly"].layout.colorway[1::-1],
    annotations=[
        dict(text='<span style="font-size:24pt;">The Myth of Police Brutality</span>',
             x=0.5,
             y=1.31,
             xref='paper', yref='paper', showarrow=False, align='center'
        ),
        dict(text='<span style="font-size:18pt;">Huge Increase in the Number of False NYPD Complaints</span>',
             x=0.5,
             y=1.16,
             xref='paper',yref='paper', showarrow=False, align='center'
        ),
        dict(
            text=('<span style="font-size:14pt">Figure 2:</span>'),
            font=(dict(color = "rgba(0,0,0,.6)")),
            x = 0, y = -0.265,
            yanchor='bottom',
            xref='paper',yref='paper', showarrow=False, align='left'
        ),
        dict(
            text=(
                '<span style="font-size:14pt">'
                'Beginning in the early 2000\'s, the NYPD began receiving an increasing number of complaints against its officers. <br>'
                'In the vast majority of complaints, the officer involved was not found to have violated department policy. By <br>'
                '2007, around 1,500 of these unsubstantiated claims were being made each year to the NYPD.</span>'
            ),
            font=(dict(color = "rgba(0,0,0,.6)")),
            x = 0.08, y = -0.4,
            yanchor='bottom',
            xref='paper',yref='paper', showarrow=False, align='left'
        ),],
    margin=dict(l=100, r=80, t=150, b=170),
)

fig.show()

pio.write_json(fig, "figures/for_nypd_misconduct.json")

In [61]:
df_nonexo = (
    dispos
    [dispos["board_disposition"] != "Exonerated"]
    .groupby("year_received")
    [["count", "percent"]]
    .sum().reset_index()
    .query("year_received>1996")
)

df_tot = (
    dispos
    .groupby("year_received")
    ["total"]
    .first()
    .reset_index()
    .query("year_received>1996")
)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_tot["year_received"],
    y=df_tot["total"],
    name="Total Complaints",
    hovertemplate="In %{x}, there were %{y} complaints.<extra></extra>"
))
fig.add_trace(go.Bar(
    x=df_nonexo["year_received"],
    y=df_nonexo['count'],
    name="Officer Not Exonerated",
    yaxis = "y2",
    #offset=-0.15,
    hovertemplate="In %{x}, %{y} officers were not exonerated.<extra></extra>"
))

fig.update_layout(
    xaxis=dict(title="Year"),
    legend=dict(title="Complaints:",
                x=0.02, xanchor='left', xref='paper',
                y=0.7, yanchor='bottom', yref='paper',
                bgcolor="rgba(255,255,255,.7)",),
    font=dict(size=18, family="Arial, sans-serif"),
    barmode="overlay",
    autosize=False,
    height = 670,
    width = 1200,
    yaxis = dict(
        title="Total Complaints",
        side='left',
        fixedrange=True),
    yaxis2=dict(
        overlaying='y', 
        range=[0,2050],
        showgrid=False,
        showticklabels=False,
        fixedrange=True),
    annotations=[
        dict(
            text='<span style="font-size:24pt;">Overwhelming Misconduct at NYPD</span>',
            x=0.5,
            y=1.31,
            xref='paper', yref='paper', showarrow=False, align='center'
        ),
        dict(
            text = '<span style="font-size:18pt;">Officers Are Rarely Exonerated When Civilians Complain</span>',
            x=0.5,
            y=1.16,
            xref='paper',yref='paper', showarrow=False, align='center'
        ),
        dict(
            text=('<span style="font-size:14pt">Figure 1:</span>'),
            font=(dict(color = "rgba(0,0,0,.6)")),
            x = 0, y = -0.265,
            yanchor='bottom',
            xref='paper',yref='paper', showarrow=False, align='left'
        ),
        dict(
            text=(
                '<span style="font-size:14pt">'
                'Beginning in the early 2000\'s, the NYPD began receiving an increasing number of complaints against its officers. <br>'
                'In these vast majority of complaints, the officer involved was not found to have been following department policy.<br>'
                'By 2007, well over 1,000 of these complaints were being made each year to the NYPD.</span>'
            ),
            font=(dict(color = "rgba(0,0,0,.6)")),
            x = 0.08, y = -0.4,
            yanchor='bottom',
            xref='paper',yref='paper', showarrow=False, align='left'
        ),
    ],
    margin=dict(l=100, r=80, t=150, b=170),
)

fig.show()

pio.write_json(fig, "figures/against_nypd_misconduct.json")

In [64]:
df_nonexo = (
    dispos
    [dispos["board_disposition"] != "Exonerated"]
    .groupby("year_received")
    [["count", "percent"]]
    .sum().reset_index()
    .query("year_received>1996")
)

df_tot = (
    dispos
    .groupby("year_received")
    ["total"]
    .first()
    .reset_index()
    .query("year_received>1996")
)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_tot["year_received"],
    y=df_tot["total"],
    name="Total Complaints",
    hovertemplate="In %{x}, there were %{y} complaints.<extra></extra>"
))
fig.add_trace(go.Bar(
    x=df_nonexo["year_received"],
    y=df_nonexo['count'],
    name="Officer Not Exonerated",
    yaxis = "y2",
    hovertemplate="In %{x}, %{y} officers were not exonerated.<extra></extra>"
))

fig.update_layout(
    xaxis=dict(title="Year"),
    legend=dict(title="Complaints:",
                x=0.02, xanchor='left', xref='paper',
                y=0.94, yanchor='top', yref='paper',
                bgcolor="rgba(255,255,255,.7)",),
    font=dict(size=16, family="Arial, sans-serif"),
    barmode="overlay",
    autosize=True,
    height=800,
    yaxis = dict(
        title="Total Complaints",
        side='left',
        fixedrange=True),
    yaxis2=dict(
        overlaying='y', 
        range=[0,2050],
        showgrid=False,
        showticklabels=False,
        fixedrange=True),
    annotations=[
        dict(
            text="Overwhelming Misconduct at NYPD",
            font=dict(size=24),
            x=0.5,
            y=1.4,
            xref='paper', yref='paper', 
            xanchor='center', yanchor='top',
            showarrow=False, align='center'
        ),
        dict(
            text = "Officers Are Rarely Exonerated When Civilians Complain",
            font=dict(size=20),
            x=0.5,
            y=1.2,
            xanchor='center', yanchor='top',
            xref='paper',yref='paper', 
            showarrow=False, align='center'
        ),
        dict(
            text=('Figure 1:'),
            font=(dict(size=16, color = "rgba(0,0,0,.6)")),
            x = 0, y = -0.465,
            xanchor='left', yanchor='bottom',
            xref='paper',yref='paper', showarrow=False, align='left'
        ),
        dict(
            text=(
                'Beginning in the early 2000\'s, the NYPD began receiving an increasing number of complaints against its officers. <br>'
                'In these vast majority of complaints, the officer involved was not found to have been following department policy.<br>'
                'By 2007, well over 1,000 of these complaints were being made each year to the NYPD.'
            ),
            font=(dict(size=16, color = "rgba(0,0,0,.6)")),
            x = 0.08, y = -0.685,
            xanchor='left', yanchor='bottom',
            xref='paper',yref='paper', showarrow=False, align='left'
        ),
    ],
    margin=dict(l=100, r=100, t=100, b=160),
)

fig.show()

pio.write_json(fig, "figures/against_nypd_misconduct.json")